### Here I use the Lstm model to can predict the next sequence of words when it takes sequence of words as input 
### This notebook is trained on the first 4 chapters of the mobydick book then use them to can predict any sequence of words you need .

### Method to can read any file as one string

In [237]:
def read_file(path):
    with open(path) as f:
        string=f.read()
    return string

### Read the first four chapters of moby dick

In [186]:
doc=read_file('moby_dick_four_chapters.txt')

## Text Cleaning

### Import spacy and tokenizer to can handel the text cleaning 

In [185]:
import spacy
import re
from keras.preprocessing.text import Tokenizer
nlp=spacy.load("en_core_web_lg")
nlp.max_length = 1198623

### This method to reomve any special char can found in the text to can only works on the words

In [189]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

### Run the method on the read document 

In [192]:
TokensInText=separate_punc(doc)

### The counts of the words in document is ??

In [193]:
len(TokensInText)

11338

### Now we need to make a sequence of 26 words to can train the model on 
### Here we said that the sequence will be 25 words and model labeled data is one word
### You can change them if you need just change the varaibles value 

In [238]:
## Now we need to make model to takes 25 words and predict the 26 word so will divid our text
length_of_taken_words=25
no_of_words_predict=1
Sentences=[TokensInText[i-(length_of_taken_words+no_of_words_predict):i] for i in range((length_of_taken_words+no_of_words_predict),len(TokensInText))]

### Example of the sentences 
### Note is is the fully sentence the 25 word plus the one word (lable)

In [239]:
Sentences[1]

['me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore']

### you can see the sentence as text

In [240]:
' '.join(Sentences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

### How many sentence we have ??

In [241]:
len(Sentences)

11312

# Now PreProcessing the data 

### How can convert the text to number ??
### Keras solve it by easy way

# Keras and Tokenizer

### We make an object of tokenizer to can takes our sentences and generate tokens

In [198]:
tokenizer=Tokenizer()

### Fit our Sentences on  the tokenizer

In [199]:
tokenizer.fit_on_texts(Sentences)

### Now our sentences are generated as number 

In [200]:
NewSentences=tokenizer.texts_to_sequences(Sentences)

### so now we can see our first sentence after converge 

In [201]:
NewSentences[0]

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 314,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2713,
 14,
 24]

### we can see how the token class convert every token to number

In [202]:
tokenizer.index_word

{1: 'the',
 2: 'a',
 3: 'and',
 4: 'of',
 5: 'i',
 6: 'to',
 7: 'in',
 8: 'it',
 9: 'that',
 10: 'he',
 11: 'his',
 12: 'was',
 13: 'but',
 14: 'me',
 15: 'with',
 16: 'as',
 17: 'at',
 18: 'this',
 19: 'you',
 20: 'is',
 21: 'all',
 22: 'for',
 23: 'my',
 24: 'on',
 25: 'be',
 26: "'s",
 27: 'not',
 28: 'from',
 29: 'there',
 30: 'one',
 31: 'up',
 32: 'what',
 33: 'him',
 34: 'so',
 35: 'bed',
 36: 'now',
 37: 'about',
 38: 'no',
 39: 'into',
 40: 'by',
 41: 'were',
 42: 'out',
 43: 'or',
 44: 'harpooneer',
 45: 'had',
 46: 'then',
 47: 'have',
 48: 'an',
 49: 'upon',
 50: 'little',
 51: 'some',
 52: 'old',
 53: 'like',
 54: 'if',
 55: 'they',
 56: 'would',
 57: 'do',
 58: 'over',
 59: 'landlord',
 60: 'thought',
 61: 'room',
 62: 'when',
 63: 'could',
 64: "n't",
 65: 'night',
 66: 'here',
 67: 'head',
 68: 'such',
 69: 'which',
 70: 'man',
 71: 'did',
 72: 'sea',
 73: 'time',
 74: 'other',
 75: 'very',
 76: 'go',
 77: 'these',
 78: 'more',
 79: 'though',
 80: 'first',
 81: 'sort',


## No of unique word in this 4 chapters

In [203]:
no_of_vocab=len(tokenizer.word_counts)

## So our new data will be this after casting to numpy array

In [232]:
NewSentences=np.array(NewSentences)

In [233]:
NewSentences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

## So first 25 will be the input and the last word will be the output 

### Our input 

In [234]:
X=NewSentences[:,:-1]

In [235]:
X

array([[ 956,   14,  263, ...,    6, 2713,   14],
       [  14,  263,   51, ..., 2713,   14,   24],
       [ 263,   51,  261, ...,   14,   24,  957],
       ...,
       [ 952,   12,  166, ...,   11,  262,   53],
       [  12,  166, 2712, ...,  262,   53,    2],
       [ 166, 2712,    3, ...,   53,    2, 2718]])

In [245]:
X.shape

(11312, 25)

### our output

In [290]:
y=NewSentences[:,-1]

In [291]:
y

array([  24,  957,    5, ...,    2, 2718,   26])

In [292]:
y.shape

(11312,)

### Now we need to convert our output to categories to (one hot encoded)

In [293]:
from tensorflow.keras.utils import to_categorical 

In [294]:
y=to_categorical(y,num_classes=no_of_vocab+1)

In [295]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### The length of the input 

In [251]:
seq_len=X.shape[1]

In [252]:
seq_len

25

# Now the time to create the model

In [187]:
import numpy as np

In [188]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Embedding
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [191]:
def create_model(vocabulary_size,seq_len):
    model=Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(150,return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150,activation='relu'))
    model.add(Dense(vocabulary_size,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [253]:
model=create_model(no_of_vocab+1,seq_len)

In [254]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 25, 25)            67975     
                                                                 
 lstm_8 (LSTM)               (None, 25, 150)           105600    
                                                                 
 lstm_9 (LSTM)               (None, 150)               180600    
                                                                 
 dense_8 (Dense)             (None, 150)               22650     
                                                                 
 dense_9 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787,394
Trainable params: 787,394
Non-trainable params: 0
_________________________________________________________________


In [255]:
from pickle import dump,load

In [181]:
model.fit(X,y,batch_size=128,epochs=20,verbose=1)

Epoch 1/20
89/89 [==============================] - 8s 89ms/step - loss: 4.5142 - accuracy: 0.1086
Epoch 2/20
89/89 [==============================] - 8s 94ms/step - loss: 4.4701 - accuracy: 0.1073
Epoch 3/20
89/89 [==============================] - 9s 99ms/step - loss: 4.4678 - accuracy: 0.1050
Epoch 4/20
89/89 [==============================] - 9s 98ms/step - loss: 4.4145 - accuracy: 0.1109
Epoch 5/20
89/89 [==============================] - 9s 104ms/step - loss: 4.3680 - accuracy: 0.1139
Epoch 6/20
89/89 [==============================] - 9s 97ms/step - loss: 4.3183 - accuracy: 0.1132
Epoch 7/20
89/89 [==============================] - 9s 98ms/step - loss: 4.2715 - accuracy: 0.1150
Epoch 8/20
89/89 [==============================] - 9s 97ms/step - loss: 4.2241 - accuracy: 0.1158
Epoch 9/20
89/89 [==============================] - 9s 100ms/step - loss: 4.1830 - accuracy: 0.1196
Epoch 10/20
89/89 [==============================] - 9s 105ms/step - loss: 4.1466 - accuracy: 0.1217
Epoch 

In [182]:
model.save("lastversion.h5")

In [183]:
dump(tokenizer, open('epochBIG', 'wb'))

## I Trained The model on GoogleColab for 300 Epoches and get about 0.99% accuracy so will use this model 

In [265]:
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
TrainedModel=keras.models.load_model("lastversion.h5")

In [266]:
tokenizer=load(open("epochBIG","rb"))

## Now Testing Time 

In [280]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind =  model.predict(pad_encoded,verbose=0)[0].argmax()
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

### Grab a random seed sequence

In [298]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [299]:
random_seed_text = Sentences[random_pick]

In [300]:
random_seed_text

['thought',
 'i',
 'to',
 'myself',
 'the',
 'man',
 "'s",
 'a',
 'human',
 'being',
 'just',
 'as',
 'i',
 'am',
 'he',
 'has',
 'just',
 'as',
 'much',
 'reason',
 'to',
 'fear',
 'me',
 'as',
 'i',
 'have']

In [301]:
generate_text(TrainedModel,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

"to be afraid of him better sleep with a sober cannibal than a drunken christian landlord said i tell him to stash his tomahawk there or pipe or whatever you call it tell him to stop smoking in short and i will turn in with him but i do n't"